# 🍁 Global Migration Trends Analysis: Canada Case Study

**Author:** Rafhiromadoni Sopandi | **Role:** Data Analyst
**Tech Stack:** Python, Pandas, Plotly Interactive
**Data Source:** United Nations - International Migration Flows

## 📌 Executive Summary
Kanada dikenal sebagai salah satu negara dengan kebijakan imigrasi paling terbuka di dunia. Dashboard ini menganalisis data historis imigrasi selama 3 dekade (1980 - 2013) untuk mengungkap pergeseran geopolitik global.

**Key Insights:**
1.  **The Asian Wave:** Pergeseran dominasi imigran dari Eropa (UK/Perancis) ke Asia (China & India) mulai tahun 1990-an.
2.  **Stability Analysis:** Negara mana yang memiliki tren pengiriman imigran paling stabil vs fluktuatif?
3.  **Global Heatmap:** Visualisasi peta dunia asal imigran.

In [1]:
# --- SYSTEM CONFIGURATION ---
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import warnings

# Supaya output bersih
warnings.filterwarnings('ignore')
print("✅ Environment Ready: Plotly Interactive Loaded.")

✅ Environment Ready: Plotly Interactive Loaded.


In [2]:
# --- DATA LOADING & PREPROCESSING ---
print("🚀 Downloading & Processing Dataset...")

# URL Dataset Publik (IBM/UN Data)
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Canada.xlsx'

# Load Excel (Sheet: Canada by Citizenship)
df = pd.read_excel(url, sheet_name='Canada by Citizenship', skiprows=20, skipfooter=2)

# 1. Cleaning Columns
# Hapus kolom metadata yang tidak perlu
df.drop(['AREA','REG','DEV','Type','Coverage'], axis=1, inplace=True)
# Rename kolom agar lebih mudah dipahami
df.rename(columns={'OdName':'Country', 'AreaName':'Continent', 'RegName':'Region'}, inplace=True)

# 2. Konsistensi Tipe Data
# Ubah nama kolom tahun menjadi string agar mudah dipanggil
df.columns = list(map(str, df.columns))
years = list(map(str, range(1980, 2014))) # List tahun 1980-2013

# 3. Set Index
df.set_index('Country', inplace=True)

# 4. Tambahkan Kolom Total
df['Total'] = df[years].sum(axis=1)

print(f"✅ Data Ready: Melacak {df.shape[0]} negara selama {len(years)} tahun.")
display(df.head(3))

🚀 Downloading & Processing Dataset...
✅ Data Ready: Melacak 195 negara selama 34 tahun.


,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,1986,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,Total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Asia,Southern Asia,Developing regions,16,39,39,47,71,340,496,...,3436,3009,2652,2111,1746,1758,2203,2635,2004,58639
Albania,Europe,Southern Europe,Developed regions,1,0,0,0,0,0,1,...,1223,856,702,560,716,561,539,620,603,15699
Algeria,Africa,Northern Africa,Developing regions,80,67,71,69,63,44,69,...,3626,4807,3623,4005,5393,4752,4325,3774,4331,69439


In [19]:
# --- 1. TREND ANALYSIS: THE RISE OF ASIA (AREA CHART) ---
import plotly.graph_objects as go

# Persiapan Data
df_ci = df.loc[['China', 'India'], years].transpose()

# Membuat Canvas
fig = go.Figure()

# Plot CHINA (Merah - Area Penuh)
fig.add_trace(go.Scatter(
    x=df_ci.index,
    y=df_ci['China'],
    mode='lines', # Hanya garis pembatas, bawahnya diisi
    name='China 🇨🇳',
    stackgroup='one', # Menumpuk data (Stacking) agar terlihat volume total
    line=dict(color='#D62728', width=2),
    fillcolor='rgba(214, 39, 40, 0.6)' # Warna Merah Solid Transparan
))

# Plot INDIA (Biru - Area Penuh)
fig.add_trace(go.Scatter(
    x=df_ci.index,
    y=df_ci['India'],
    mode='lines',
    name='India 🇮🇳',
    stackgroup='one',
    line=dict(color='#1F77B4', width=2),
    fillcolor='rgba(31, 119, 180, 0.6)' # Warna Biru Solid Transparan
))

# Kustomisasi Layout (AGAR MELEBAR & JELAS)
fig.update_layout(
    # 1. Pengaturan Judul
    title=dict(
        text='<b>The Rise of Asian Giants</b><br><i>Cumulative Immigration Volume (1980-2013)</i>',
        x=0.5, # Judul Rata Tengah (Center)
        xanchor='center',
        y=0.95 # Posisi vertikal judul (sedikit turun biar gak kena batas atas)
    ),
    title_font_size=22,

    xaxis_title='Tahun',
    yaxis_title='Jumlah Imigran (Kumulatif)',
    template='plotly_white',

    # 2. Ukuran Canvas
    height=600,
    # Margin diperluas: Atas (t) jadi 120 agar judul 2 baris muat
    margin=dict(l=40, r=40, t=120, b=80),

    hovermode="x unified",

    # 3. Posisi Legenda DIPINDAH KE BAWAH (Agar tidak nabrak judul)
    legend=dict(
        orientation="h", # Horizontal
        yanchor="top",
        y=-0.15, # Di bawah sumbu X
        xanchor="center",
        x=0.5 # Rata tengah
    )
)

# Anotasi Konteks
fig.add_annotation(
    x="2005", y=70000, # Posisi di tengah area
    text="Dominasi Total > 70k/tahun",
    showarrow=False,
    font=dict(size=14, color="black")
)

fig.show()

In [6]:
# --- 2. GLOBAL MIGRATION HEATMAP ---
# Reset index agar 'Country' bisa dibaca oleh peta
df_map = df.reset_index()

fig = px.choropleth(df_map,
                    locations="Country",
                    locationmode='country names', # Mencocokkan nama negara dengan peta dunia
                    color="Total",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Peta Asal Imigran ke Kanada (Total 1980-2013)",
                    labels={'Total':'Jumlah Imigran'})

fig.update_layout(geo=dict(showframe=False, showcoastlines=False))
fig.show()

In [7]:
# --- 3. TOP 5 CONTRIBUTOR COUNTRIES ---
# Ambil 5 negara teratas berdasarkan Total
df_top5 = df.sort_values(by='Total', ascending=False).head(5)
df_top5 = df_top5[years].transpose() # Transpose agar tahun jadi sumbu X

fig = px.area(df_top5, x=df_top5.index, y=df_top5.columns,
              title='Kontribusi Kumulatif 5 Negara Terbesar',
              labels={'index':'Tahun', 'value':'Jumlah Imigran', 'variable':'Negara'})

fig.show()

In [21]:
# --- 4. HIERARCHY ANALYSIS (SUNBURST CHART) ---
# Menyiapkan data tahun 2013 saja (Snapshot terakhir)
df_2013 = df.reset_index()[['Continent', 'Region', 'Country', '2013']]

# Filter out rows where the '2013' value is 0 to avoid ZeroDivisionError
df_2013 = df_2013[df_2013['2013'] > 0]

fig = px.sunburst(df_2013,
                  path=['Continent', 'Region', 'Country'],
                  values='2013',
                  title='Hierarki Asal Imigran (Tahun 2013)',
                  color='2013',
                  color_continuous_scale='RdBu')

fig.update_layout(height=700)
fig.show()

## 💡 Key Insights & Conclusion

1.  **Shift of Power:** Data menunjukkan pergeseran tren migrasi yang masif. Hingga tahun 1980-an, migrasi didominasi Eropa, namun sejak 1990-an, **China dan India** mengambil alih secara signifikan.
2.  **Economic Correlation:** Lonjakan imigran dari Asia pada akhir 1990-an berkorelasi kuat dengan kebijakan Kanada dalam merekrut tenaga kerja sektor teknologi (*IT Boom*).
3.  **Stability:** Meskipun China dan India mendominasi volume, negara seperti **Filipina** menunjukkan tren pertumbuhan yang paling stabil (kurva mulus naik) dibandingkan fluktuasi tajam dari negara lain.